In [1]:
import torch
import torch.nn as nn

from SepFormer import SepFormerSeparator
from cRFConvTasNet import TCN

/home/kbh/.conda/envs/dnn/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


```python
    def __init__(self, N=64, C=2, L=4, H=4, K=250, Global_B=2, Local_B=4):

        super(Sepformer, self).__init__()

        self.N = N  # n_encoder_output_channel 
        self.C = C  # n_source
        self.L = L  # size of encoder conv 
        self.H = H  # be careful for n_head
        self.K = K  # size of chunk
        self.Global_B = Global_B  # n_global_loop
        self.Local_B = Local_B  # n_local_loop

        self.encoder = Encoder(self.L, self.N)

        self.separator = SepFormerSeparator(self.N, self.C, self.H, self.K, self.Global_B, self.Local_B)

        self.decoder = Decoder(self.L, self.N)
```

In [2]:
f_ch = 256
dim_input = 6168

In [3]:
m0  = nn.Conv1d(dim_input,f_ch,1)

m1 = TCN(
    c_in = f_ch,
         c_out= f_ch*2,
        kernel = 3,
            n_successive =2,
            n_block = 8,
            TCN_activation="None"
        )
m2 = SepFormerSeparator(f_ch,1,4,250,2,4)
dim_output = 4 * 4 * 257 * ((2*1+1)*(2*1+1)) * 2

m3 = nn.Conv1d(
    f_ch,
    dim_output,
    1
    )



In [4]:
x = torch.rand(2,6168,500)
print(x.shape)

x1 = m0(x)
print(x1.shape)

x2 = m1(x1)
print(x2.shape)

x3 = m2(x2)
print(x3.shape)

x4 = m3(x3)
print(x4.shape)

filter = torch.reshape(x4,(2,4,4,3,3,257,500,2))
print(filter.shape)

torch.Size([2, 6168, 500])
torch.Size([2, 256, 500])
torch.Size([2, 256, 500])
torch.Size([2, 256, 500])
torch.Size([2, 74016, 500])
torch.Size([2, 4, 4, 3, 3, 257, 500, 2])


### 